<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/ce/Huberlin-logo.svg/200px-Huberlin-logo.svg.png" alt="fff" align="left" style="width: 150px;width: 150px;align:left">


# Recurrent Encoder-Decoder Model (RED)
 
#### Thesis: Neural response generation for email conversations
#### Author: Sydney Richards
#### Required Runtime: CPU
#### Model: tf-idf similarity





This notebook implements th tf-idf benchmark model which matches templates to input sequences. The data pre-processing is the same as in the neural models. his workbook can be used for both the English as well as the German data set. However, the default set up is for the English data otherise all references to the dataframe "data" have to be replaced with "dataDE".


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
#import tensorflow_hub as hub
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
import datetime as dt
import pandas as pd


seed = np.random.RandomState(seed=10)

TensorFlow 2.x selected.


### Stanford NER Tagger

Stanford NER is a Java implementation of a Named Entity Recognizer. Named Entity Recognition (NER) labels sequences of words in a text which are the names of things, such as person and company names, or gene and protein names. It comes with well-engineered feature extractors for Named Entity Recognition, and many options for defining feature extractors. Included with the download are good named entity recognizers for English and German, particularly for the 3 classes (PERSON, ORGANIZATION, LOCATION).

We use the Tagger to itdentify names and replace them with tokens, since we do not wan't our model to have to remember names just add a token at the position of the sentence were a name should appear. 

[Stanford NLP Reference](https://nlp.stanford.edu/software/CRF-NER.shtml)

In [0]:
!pip install stanfordcorenlp

In [0]:
!pip install nltk --upgrade
!pip install rouge
import nltk
nltk.download('wordnet')


!wget 'https://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05'
!unzip stanford-corenlp-full-2018-10-05

nltk.download('punkt')



     |████████████████████████████████| 1.5MB 2.7MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449903 sha256=0df36405ad8234096a27a50b20a89273f1c01fd5b6ab0dc19bd8519ddbf08786
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


--2020-02-05 10:55:22--  https://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 404 NOT FOUND
2020-02-05 10:55:23 ERROR 404: NOT FOUND.

unzip:  cannot find or open stanford-corenlp-full-2018-10-05, stanford-corenlp-full-2018-10-05.zip or stanford-corenlp-full-2018-10-05.ZIP.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
## text = 'Guangdong University is eine Auslandsschule. ' \
##       'Mein Gott der Niklas is ja dick geworden. '

##props={'annotators': 'ner','pipelineLanguage':'de','outputFormat':'json'}


In [0]:
from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


### Load and Merge Datasets

Data sets are tables of e-mail activities where each activity represents a single mail either from customer to the company or vice versa. In order to link
customer enquiries and customer support responses we use the Case ID which is an unique ID given to a e-mail thread of a customer. Each customer can have multiple threads or cases and each case can have mutiple e-mails from both the customer as well as the company but the thread will usually deal with a single issue e.g. "Could ou please check if I have outstanding bills on my  acccount." 

In [0]:
### Self-Join tables over Case_ID 

## EN
activities = pd.read_csv('/content/drive/My Drive/Master Thesis/act_in_out_API.csv') 
activities = pd.merge(activities.loc[activities['ACTIVITY_MODE']=='inbound',['CASE_ID','CONTENT_TEXT','CLASSIFICATIONS']],
                      activities.loc[activities['ACTIVITY_MODE']=='outbound',['CASE_ID','CONTENT_TEXT','CLASSIFICATIONS']],
                      how='inner',left_on='CASE_ID',right_on = 'CASE_ID')

activities2 = pd.read_csv('/content/drive/My Drive/Master Thesis/act_in_out.csv')
activities2 = pd.merge(activities2.loc[activities2['ACTIVITY_MODE']==100,['CASE_ID','CONTENT_TEXT']],
                       activities2.loc[activities2['ACTIVITY_MODE']==200,['CASE_ID','CONTENT_TEXT']],
                       how='inner',left_on='CASE_ID',right_on = 'CASE_ID')

data = pd.concat([activities,activities2],sort=False)
data = data.loc[(data['CONTENT_TEXT_x'].isnull()==False)&(data['CONTENT_TEXT_y'].isnull()==False),:]
data = data.reset_index(drop=True)
data = data.rename({'CONTENT_TEXT_x':'Incoming','CONTENT_TEXT_y':'Outgoing'},axis=1)

## DE
activities_de = pd.read_csv('/content/drive/My Drive/Master Thesis/act_in_out_API_de.csv')
activities_de = pd.merge(activities_de.loc[activities_de['ACTIVITY_MODE']=='inbound',['CASE_ID','CONTENT_TEXT','CLASSIFICATIONS']],
                         activities_de.loc[activities_de['ACTIVITY_MODE']=='outbound',['CASE_ID','CONTENT_TEXT','CLASSIFICATIONS']],
                         how='inner',left_on='CASE_ID',right_on = 'CASE_ID')

dataDE = activities_de.reset_index(drop=True)
dataDE = dataDE.rename({'CONTENT_TEXT_x':'Incoming','CONTENT_TEXT_y':'Outgoing'},axis=1)
dataDE = dataDE.reset_index(drop=True)

## KB

kb = pd.read_csv('/content/drive/My Drive/Master Thesis/KB_articles.csv')


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [0]:
kb.shape

(8398, 2)

In [0]:
kb = kb.loc[kb['ARTICLE_NAME'].str.find('?')>0,:]

In [0]:
def print_overview(dataset):
  # Identify number of missing cases if only a small amount drop
  print('Data set has %s records' % "{:,.0f}".format(dataset.shape[0]))
  print('%s of records have missings' % "{0:.2f}%".format((1-(dataset.dropna().shape[0])/dataset.shape[0]) * 100))
  print(dataset.isnull().sum())

  # Show some discriptive statistics on dataset
  print('{:^40}'.format('Column Name'),'','{:10}'.format('Unique Values'),'{:^50}'.format('Examples'))
  for x in list(dataset.columns.values):
    print('{:^40}'.format(x),'','{:10d}'.format(len(dataset[x].unique())),'{:^50}'.format(str(dataset[x].unique()[:3])[:35]))


In [0]:
print_overview(data)

Data set has 149,492 records
9.42% of records have missings
CASE_ID                  0
Incoming                 0
CLASSIFICATIONS_x       15
Outgoing                 0
CLASSIFICATIONS_y    14079
dtype: int64
              Column Name                 Unique Values                      Examples                     
                CASE_ID                       120265               [116483 116582 116279]              
                Incoming                      125628        ['Description of issue: hi there --        
           CLASSIFICATIONS_x                    1824        ['DO NOT USE - Cleaning[**********]        
                Outgoing                      128585        ["Hi Ashley,\n\nThank you for conta        
           CLASSIFICATIONS_y                    1690        ['DO NOT USE - Cleaning[**********]        


In [0]:
print_overview(dataDE)

Data set has 148,898 records
6.14% of records have missings
CASE_ID                 0
Incoming              765
CLASSIFICATIONS_x      11
Outgoing                1
CLASSIFICATIONS_y    8456
dtype: int64
              Column Name                 Unique Values                      Examples                     
                CASE_ID                       118119               [25309. 33872. 56127.]              
                Incoming                      125049        ['Sehr geehrte Damen und Herren, da        
           CLASSIFICATIONS_x                    1794        ['346 Unblocking Request  - Amount         
                Outgoing                      127519        ['Hallo Stefanie,\nvielen Dank für         
           CLASSIFICATIONS_y                    1690        ['346 Unblocking Request  - Amount         


Split E-Mail message body into individual sequences in order to clean the content from unneccessary text especially response history, headers and footers.


###### List of split symbols to extract message

|Incoming|Outgoing|
|-----|--------|
|\r\n\r\nSent|\n\n\xa0\n|
|\n\nOn|-|
|\n\n\n|-|
|\r\n\r\n\r\n|-|
|\r\nOn|-|

In [0]:
data['Outgoing'] = data['Outgoing'].str.replace('Thank you for contacting car2go!','')

In [0]:
dataDE['Outgoing'] = dataDE['Outgoing'].str.replace('vielen Dank für deine E-Mail','') 
dataDE['Outgoing'] = dataDE['Outgoing'].str.replace('vielen Dank für deine Rückmeldung','')

In [0]:
def find_split(x):
    # x: Incoming E-Mail Message
    # split_ops: list of for which string should be split
    # Idea split point should be anywhere after 5% or before 90% of the text l
    split_ops = ['\r\n\r\nSent','\r\nOn','\r\n\r\n\r\n','\r\nFrom:']
    for op in split_ops:
        if (x.find(op)>=len(x) * 0.05)&(x.find(op)<=len(x) * 0.9):
            return x[:x.find(op)]
    
    return x

In [0]:
FOOTER = ['T 1 877\xa0488 4224\nmemberservicesNA@car2go.com\nwww.car2go.com\ncar2go NA, LLC\n1717 West 6th Street, Suite #425\nAustin, Texas 78703\nCAR2GO – PROUD TO SHARE\nDisclaimer:\nThis e-mail may contain confidential information. If you are not the addressee you are not authorized to make use of the information contained in this e-mail. Please inform us immediately that you have received it by mistake.\n\n\n'
,'T 1 877 488 4224\nmemberservicesNA@car2go.com\nwww.car2go.com\n\nCAR2GO – PROUD TO SHARE\n\nDisclaimer:\nThis e-mail may contain confidential information. If you are not the addressee you are not authorized to make use of the information contained in this e-mail. Please inform us immediately that you have received it by mistake.\n\n',
         '\nT 1 855 454 1002\nmemberservicesCAN@car2go.com\nwww.car2go.com\n\ncar2go Canada Ltd.\n321 Water St, Suite #330\nVancouver, BC V6B 1B8\n\nCAR2GO – PROUD TO SHARE\n\nDisclaimer:\nThis e-mail may contain confidential information. If you are not the addressee you are not authorized to make use of the information contained in this e-mail. Please inform us immediately that you have received it by mistake.',
         '\nT 1 855 454 1002\nmemberservicesCAN@car2go.com\nwww.car2go.com\n\ncar2go Canada Ltd.\n1717\xa0West 6th St #425\nAustin, Texas 78703\n\nCAR2GO – PROUD TO SHARE\n\nDisclaimer:\nThis e-mail may contain confidential information. If you are not the addressee you are not authorized to make use of the information contained in this e-mail. Please inform us immediately that you have received it by mistake.\n',
         "\n\n\n\nIf you have any additional questions or concerns, please don't hesitate to let me know."
          ,"\nWas my solution helpful? We'd love to hear your feedback! Share your experience with us by taking part in our\xa0survey.\n",
         "\n\n\n\n\nIf you have any additional questions or concerns, please don't hesitate to contact us.",
         "Kundenservice\n\n\xa0\n\nCAR2GO – PROUD TO SHARE\n\n\xa0\n\ncar2go Deutschland GmbH\n\nAnschrift/postal address: car2go Deutschland GmbH | HPC HB20 | 10875 Berlin | Deutschland\nRegistergericht/Court of Registry: Amtsgericht Stuttgart\nHRB-Nr./Commercial Register No.: 737715\nGeschäftsführer/Board of Directors: Olivier Reppert, Stefan Glebke\n\nDisclaimer:\nDiese Mail kann vertrauliche Informationen enthalten. Wenn Sie nicht Adressat sind, sind Sie nicht zur Verwendung der in dieser Mail enthaltenen Informationen befugt. Bitte benachrichtigen Sie uns sofort über den irrtümlichen Empfang.\nThis e-mail may contain confidential information. If you are not the addressee you are not authorized to make use of the information contained in this e-mail. Please inform us immediately that you have received it by mistake.",
         "\nWas my solution helpful? We'd love to hear your feedback! Share your experience with us\xa0by taking\xa0part in our SURVEY.\n\nT 1 855 454 1002\nmemberservicesCAN@car2go.com\nwww.car2go.com\n\ncar2go Canada Ltd.\n1717\xa0West 6th St #425\nAustin, Texas 78703\n\nCAR2GO – PROUD TO SHARE\n\nDisclaimer:\nThis e-mail may contain confidential information. If you are not the addressee you are not authorized to make use of the information contained in this e-mail. Please inform us immediately that you have received it by mistake.",
         "Was my solution helpful? We'd love to hear your feedback! Share your experience with us\xa0by taking\xa0part in our SURVEY.",
         '\nT 1 855 454 1002\n\nmemberservicesCAN@car2go.com\n\nwww.car2go.com\n\ncar2go Canada Ltd.\n\n321 Water St, Suite #330\n\nVancouver, BC V6B 1B8\n\nCAR2GO – PROUD TO SHARE\n\nDisclaimer:\n\nThis e-mail may contain confidential information. If you are not the addressee you are not authorized to make use of the information contained in this e-mail. Please inform us immediately that you have received it by mistake.']

In [0]:
def split_out(x):
    # x: Outgoing E-Mail Message
    # Outgoing Messages always contain a similar content 
    x  = x[:x.find('--Ursprüngliche Nachricht--')]
    x = x[x.find(',')+1:]
    for f in FOOTER:
        x = x.replace(f,'')
    
    
    return '.'.join(x.split('\n\n\xa0\n')[0].split('.')[:2])

In [0]:
def name_tokenizer(x):
  for token in nlp.ner(x):
    if token[1] == 'PERSON':
      x = x.replace(token[0],'<name>')
  
  return x

In [0]:
def cash_tokenizer(x):
    #x text
    if x.find('$') > 0:
        return x.replace(x[x.find('$'):x.find('$') + x[x.find('$'):].find(' ')],'<amt>')
    elif x.find('€') > 0:
        return x.replace(x[x.find('€'):x.find('€') + x[x.find('€'):].find(' ')],'<amt>')
    return x

In [0]:
def prep_outging(x):
    x = split_out(x)
    x = cash_tokenizer(x)
    #x = name_tokenizer(x)
    
    return x

In [0]:
#data.loc[~data['Outgoing'].str.count('<amt>') > 4,:]

In [0]:
incomings = []
outgoings = []
#data = data.sample(50000,random_state=seed).reset_index(drop=True)
for row_no, row in data.iterrows():
    if row_no in range(0,data.shape[0],10000):
        print('...%s lines converted out of %s' % ("{:,}".format(row_no),"{:,}".format(data.shape[0])))
         
    incomings.append(find_split(row['Incoming']))
    outgoings.append(prep_outging(row['Outgoing']))


...0 lines converted out of 149,492
...10,000 lines converted out of 149,492
...20,000 lines converted out of 149,492
...30,000 lines converted out of 149,492
...40,000 lines converted out of 149,492
...50,000 lines converted out of 149,492
...60,000 lines converted out of 149,492
...70,000 lines converted out of 149,492
...80,000 lines converted out of 149,492
...90,000 lines converted out of 149,492
...100,000 lines converted out of 149,492
...110,000 lines converted out of 149,492
...120,000 lines converted out of 149,492
...130,000 lines converted out of 149,492
...140,000 lines converted out of 149,492


In [0]:
data['enquiry'] = incomings
data['reply'] = outgoings

In [0]:
### ISSSUE: To many <amt> signs after lost 5000 data points 
data = data.loc[data['reply'].str.count('<amt>')<5,:]

In [0]:
data = data.loc[(data['enquiry'].str.len()>=50)&(data['reply'].str.len()>=50),:]
data = data.reset_index(drop=True)
data.shape

(112410, 7)

In [0]:
def clean_classification(x):
  try:
    return re.sub(r"[^a-zA-Z&; ]+", "",x).replace('deleted','').replace('cargo','<company>').strip(' ')
  except:
    return x
    
    
data['CLASSIFICATIONS'] = [clean_classification(x) for x in data['CLASSIFICATIONS_x']]

In [0]:
class pd_frame_to_tf_tensor_pipe():
  def __init__(self,inputs,targets,Max_Length = 50,train_size=0.2):
    self._inputs = inputs
    self._targets = targets
    self._Max_Length = Max_Length
    self._train_size = train_size
  def _preprocess_sentence(self,w,is_input=False):
      #w = unicode_to_ascii(w.lower().strip())

      # creating a space between a word and the punctuation following it
      # eg: "he is a boy." => "he is a boy ."
      # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
      w = w.lower().strip()
      w = re.sub(r"([?.!,¿])", r" \1 ", w)
      w = re.sub(r'[" "]+', " ", w)
      w = w.replace('\n', ' ').replace('\r', ' ').replace('\xa0',' ')
      if is_input == True:
      # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
        w = re.sub(r"[^a-zA-Z?.!,0-9]+", " ", w)
      w = w.replace('car2go','<company>')
      w = w.replace('Share Now','<company>')

      w = w.rstrip().strip()

      # adding a start and an end token to the sentence
      # so that the model know when to start and stop predicting.
      w = '<start> ' + w + ' <end>'
      return w

  def _tokenize(self,corpus):
      tokenizer = tf.keras.preprocessing.text.Tokenizer(
          filters='')
      tokenizer.fit_on_texts(corpus)

      tensor = tokenizer.texts_to_sequences(corpus)
      #tensor = [len(tokenizer.word_index)] + tensor + [len(tokenizer.word_index + 1)]
      #tensor = tf.convert_to_tensor(tensor)
      tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                            padding='post',
                                                            maxlen=self._Max_Length)

      return tensor,tokenizer
  
  def get_tensors(self):
      enquiries = [self._preprocess_sentence(x,True) for x in self._inputs]
      replies = [self._preprocess_sentence(x) for x in self._targets]

      input_tensor, input_tokenizer = self._tokenize(enquiries)
      target_tensor, target_tokenizer = self._tokenize(replies)
      
      # Calculate max_length of the target tensors
      self.max_length_targ, self.max_length_inp = max([len(x) for x in target_tensor]),max([len(x) for x in input_tensor])

      input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)
      
      self.len_inp_train = len(input_tensor_train)
      
      train_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(len(input_tensor_train))
      
      test_dateset = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val)).shuffle(len(input_tensor_val))

      return train_dataset, test_dateset, input_tokenizer, target_tokenizer


In [0]:
pipeline = pd_frame_to_tf_tensor_pipe(inputs=data['enquiry'].to_list()
                                      ,targets = data['reply'].to_list())

In [0]:
kb = kb.reset_index(drop=True)

In [0]:
kb['articles'] = [pipeline._preprocess_sentence(x) for x in kb['CONTENT_TEXT'].astype(str)]

In [0]:
kb['articles'] = kb['articles'].str.replace('<start> ','')
kb['articles'] = kb['articles'].str.replace('<end>','')

In [0]:
data['Inputs'] = [pipeline._preprocess_sentence(x,True) for x in data['Incoming'].astype(str)]
data['Inputs'] = data['Inputs'].str.replace('<start> ','')
data['Inputs'] = data['Inputs'].str.replace('<end>','')

In [0]:
data['Targets'] = [pipeline._preprocess_sentence(x,True) for x in data['Outgoing'].astype(str)]
data['Targets'] = data['Targets'].str.replace('<start> ','')
data['Targets'] = data['Targets'].str.replace('<end>','')

In [0]:
train_data,test_data =  train_test_split(data, test_size=0.2)

In [0]:
from gensim import  corpora, models, similarities

In [0]:
text = train_data['Inputs'].to_list() + kb['articles'].to_list()

In [0]:
text = [t.split(' ') for t in text]

In [0]:
dictionary = corpora.Dictionary(text)
feature_cnt = len(dictionary.token2id)

In [0]:
kb_articles = [article.split(' ') for article in kb['articles'].to_list()]

In [0]:
corpus = [dictionary.doc2bow(t) for t in kb_articles]
tfidf = models.TfidfModel(corpus)
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = feature_cnt)

In [0]:
def match_KB_article(input):
  input = input.split(' ')
  kw_vector = dictionary.doc2bow(input)
  sim = index[tfidf[kw_vector]]
  return kb.loc[np.argmax(sim),'articles']

In [0]:
test_data['KB_Article'] = [match_KB_article(x) for x in test_data['Inputs']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
print(match_KB_article('Please cancel my account and remove all credit card information from your system.'))

i’m sorry to hear you’re having trouble connecting to the vehicle’s bluetooth .  to make room for your phone , you can remove a device by using the control knob to scroll to “settings” and then to “bluetooth” . you can then view a list of connected devices , and select the device you’d like to remove by going to “options” and then “de-authorize” . 


In [0]:
print(match_KB_article('Please see attached copy of my drivers license for my Car2Go records.'))

it's great that you inform us about your new name .  [married] congratulations to you and your new partner ! i'll be happy to help make it official with the change in your account .  updating your information is easy ! attached you will find the name change form . please send us these documents: 	 filled in name change form	 a copy of your marriage or name change certificate or a copy of the front side of your id .  you can just answer to this email and attach all necessary documents . 


In [0]:
print(match_KB_article('Hi there, Please find my proof of refuel purchase along with the car2go rental it applies to below.'))

we appreciate you taking the time to refuel our vehicle . thanks for submitting your receipt ! i’ve refunded you the cost of your refuel (xxxx) . you can expect to see this reflected on your credit card statement in the next few business days .  keep in mind our vehicles are very fuel efficient and with a fuel level of 10% , you can drive approximately 25 miles . so in most cases , $5 will be enough to get you where you're going .  if you must refuel , please be sure to add supreme/premium gas (91 octane) only . 


In [0]:
print(match_KB_article('Car rental today - vehicle had warning light on that tire sensor was not working'))

i had a look at your rental from dd . mm . xxx with the x-go xxx and i can see that the rent wasn't finished properly , because option 1 (door): the <company> the driver's door the passenger door the trunk door was not locked .  option 2 (keys , cards): the key the fuel card the parking card was not in the holder provided for it .  option 3 (business area): you wanted to rent the <company> outside the business area . since it is not possible , your rental continued .  we also informed you about the end of rental error via push message in the <company> app . if you did not receive our message , please check your settings .  we have finished the rental for you .  you will have finished the rental at a later date .  your rental continued and the doors were locked at the end of the rental period . during the rental you are responsible for possible misuse of others , so please do always make sure that the requirements for ending the rental are fulfilled . just wait until the hazard lights l

In [0]:
print(match_KB_article('Hello, do you have a lost and found for items left in cars? I lost a scarf and Cab which are quite sentimental to me.'))

oh no ! i’m sorry you lost your xxxx .  all items we find in our vehicles are posted on <company> . crowdfind . com . there you can browse through the lost items , and if you come across your xxxx , file a claim with the crowdfind team , and they’ll send it to you .  if you don’t find what you’re looking for , don’t worry , you can still file a claim , and they’ll contact you if your xxxx does turn up . 


- Please cancel my account and remove all credit card information from your system.
- Please see attached copy of my drivers license for my Car2Go records.
- Hi there, Please find my proof of refuel purchase along with the car2go rental it applies to below.
- Car rental today - vehicle had warning light on that tire sensor was not working
- Hello, do you have a lost and found for items left in cars? I lost a scarf and Cab which are quite sentimental to me.
- Why am I not able to end my rental?

In [0]:
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
from nltk.translate.meteor_score import meteor_score
calc_rouge = Rouge()

test_data = test_data.reset_index(drop = True)
chencherry = nltk.translate.bleu_score.SmoothingFunction()
bleus = []
rouges = []
meteors = []
for no,row in test_data.iterrows():
  a = ' '.join(row['KB_Article'].split(' ')[:50])
  b = ' '.join(row['Targets'].split(' ')[:50])
  if no % 1000 == 0:
    print('Test Row Number {}'.format(no))
  bleus.append(sentence_bleu([b.split(' ')],
                                a.split(' '),smoothing_function=chencherry.method1))
 
  meteors.append(meteor_score(b,a))
  scores = calc_rouge.get_scores(b,a)
  rouges.append(scores[0]['rouge-1']['f'])

df_result = pd.DataFrame({'BLEU':bleus,'ROUGE-1':rouges,'METEOR':meteors,'Enquiry':test_data['Inputs'].to_list(),'Bot':test_data['KB_Article'].to_list(),'True':test_data['Targets'].to_list()})

Test Row Number 0
Test Row Number 1000
Test Row Number 2000
Test Row Number 3000
Test Row Number 4000
Test Row Number 5000
Test Row Number 6000
Test Row Number 7000
Test Row Number 8000
Test Row Number 9000
Test Row Number 10000
Test Row Number 11000
Test Row Number 12000
Test Row Number 13000
Test Row Number 14000
Test Row Number 15000
Test Row Number 16000
Test Row Number 17000
Test Row Number 18000
Test Row Number 19000
Test Row Number 20000
Test Row Number 21000
Test Row Number 22000


In [0]:
df_result['CLASSIFICATION'] = test_data['CLASSIFICATIONS']

In [0]:
df_result.to_csv('/content/drive/My Drive/Master Thesis/Results/baseline.csv')

In [0]:
for c in df_result['CLASSIFICATION'].value_counts()[:7].index:
     print('Results for {} \nBLEU: {:.2f} ROUGE-1: {:.2f} METEOR: {:.2f}'.format(c,df_result.loc[df_result['CLASSIFICATION'] ==c,'BLEU'].mean() *100
                                                                        ,df_result.loc[df_result['CLASSIFICATION'] ==c,'ROUGE-1'].mean()*100
                                                                        ,df_result.loc[df_result['CLASSIFICATION'] ==c,'METEOR'].mean()*100))



Results for Account Balance 
BLEU: 7.55 ROUGE-1: 25.64 METEOR: 9.46
Results for Manage Member Account 
BLEU: 3.35 ROUGE-1: 20.98 METEOR: 9.51
Results for Outstanding Balance 
BLEU: 5.18 ROUGE-1: 24.78 METEOR: 9.23
Results for Validation of DL 
BLEU: 1.28 ROUGE-1: 18.11 METEOR: 10.74
Results for System Only Cleanup 
BLEU: 2.66 ROUGE-1: 20.89 METEOR: 9.81
Results for Account Deactivation 
BLEU: 5.74 ROUGE-1: 22.75 METEOR: 9.07
Results for International DL 
BLEU: 4.03 ROUGE-1: 21.57 METEOR: 8.85


In [0]:
for c in class_results.keys():
  print(c +'\n',class_results[c].mean())

Account Balance
 BLEU       0.034524
ROUGE-1    0.215427
METEOR     0.095082
dtype: float64
Manage Member Account
 BLEU       0.036568
ROUGE-1    0.217811
METEOR     0.095245
dtype: float64
Outstanding Balance
 BLEU      NaN
ROUGE-1   NaN
METEOR    NaN
dtype: float64
Validation of DL
 BLEU       0.028840
ROUGE-1    0.210003
METEOR     0.093860
dtype: float64
Account Deactivation
 BLEU       0.044005
ROUGE-1    0.224749
METEOR     0.094815
dtype: float64
